# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Question 1

### Retrieval of Toronto postcode information + cleaning and reshaping

The following notebook is the answer of the first question in the Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto.

The steps are:
1. importing necessary libraries.
1. Requesting the wikipedia page that containt the Neighboorhood info for Toronto.
1. Parsing the page with BeautifulSoup to extract the table
1. Creating the dataframe
1. Applying the requested modification for the dataframe


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

#### Retrieval of the web page + transformation into a BeautifulSoup object

In [2]:
webcontent = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(webcontent,'lxml')

#### Storage of the header in a list, and the table content in a dictionary
a replace is added in the 3 column to remove the \n that is present in the raw data received

In [3]:
postaltable = soup.find('table')
postaltablecol = []
postaltablecontent = {}

for i, row in enumerate(postaltable.find_all('tr')):
    if i == 0:
        for Cval in row.find_all('th'):
            postaltablecol.append(Cval.text.replace('\n',''))
    else:
       postaltablecontent[i] =[row.find_all('td')[0].text ,  row.find_all('td')[1].text,  row.find_all('td')[2].text.replace('\n','')]

#### Creation of the first version of the dataframe and a reset of the index

In [4]:
df = pd.DataFrame.from_dict(postaltablecontent,orient = 'index',columns = postaltablecol)
df.reset_index(inplace = True, drop = True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Removal of the rows with empty Borough and change of the value "Not assigned" of Neighbourhood by the Borough value.
the change of value was initially done with numpy, but was after change with the equivalent in Pandas, the numpy version was kept for future reference.


In [5]:
df = df[df['Borough']!= 'Not assigned']
#df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'],df['Neighbourhood'])
df['Neighbourhood'].where(df['Neighbourhood'] != 'Not assigned',df['Borough'], inplace = True)
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### Last step which is to regroup the Neighbourhood that have the same Postcode using DataFrame.groupby() .
The Unique() method is first used to list the Neighbourhood values and then are transform into a string  with join.

In [6]:
df = df.groupby(['Postcode','Borough']).Neighbourhood.unique().reset_index()
df['Neighbourhood']= df['Neighbourhood'].apply(', '.join)

df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### The shape of the final datafrape is as follow

In [7]:
print(df.shape)

(103, 3)
